# Data Mining project
## Members:
- Mateusz Idziejczak 155842
- Mateusz Stawicki 155900
- Anastasiya
- Baris
## Dataset
The dataset we are going to use is the [Titanic](https://www.kaggle.com/competitions/titanic) dataset from Kaggle. It contains data of 2224 passengers. The dataset is in a CSV format and contains TODO columns. The columns are: TODO

## Implementation
### Data loading
First we need to load the data from the CSV file. We will use the pandas library to do this.


In [353]:
import pandas as pd

data = pd.read_csv('data/train.csv')

### Data exploration
Now we will explore the data to see what it contains.

In [354]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [355]:
data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


### Data preprocessing
Before we can start building the model we need to preprocess the data. This includes:
- splitting the data into features and labels
- Removing columns that are not useful
- Handling missing values
- Encoding categorical variables
- Scaling the data
- Splitting the data into training and testing sets

#### Splitting the data into features and labels

In [356]:
X = data.drop('Survived', axis=1)
y = data['Survived']

#### Removing columns that are not useful
We will remove the `PassengerId`, `Name` and `Ticket` columns as they are not useful for the model.

In [357]:
X = X.drop(['PassengerId', 'Name', 'Ticket'], axis=1)

#### Handling missing values
We will fill the missing values in the `Age` and `Cabin` columns with the mean and mode respectively.

In [358]:
X['Age'] = X['Age'].fillna(X['Age'].mean())
X['Cabin'] = X['Cabin'].fillna(X['Cabin'].mode()[0])

#### Encoding categorical variables
We will encode the `Embarked` and `Sex` column using one-hot encoding.

In [359]:
columns_to_encode = ['Embarked', 'Sex']
try:
    X = pd.get_dummies(X, columns=columns_to_encode)
except KeyError:
    print('Columns already encoded')
# We will also encode the `Cabin` column using label encoding
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
X['Cabin'] = label_encoder.fit_transform(X['Cabin'])
# or simply delete cabin column because there are too many missing values and unique values
# X = X.drop('Cabin', axis=1)

#### Scaling the data
We will scale the data using the L1 normalization.

In [360]:
from sklearn.preprocessing import Normalizer

scaler = Normalizer(norm='l1').fit(X)
X = pd.DataFrame(scaler.transform(X), columns=X.columns)

#### Splitting the data into training and testing sets
We will split the data into training and testing sets using the `train_test_split` method from the `sklearn` library.

In [361]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

#### Final data
The final data looks like this:

In [362]:
X.head()

,Pclass,Age,SibSp,Parch,Fare,Cabin,Embarked_C,Embarked_Q,Embarked_S,Sex_female,Sex_male
0,0.036474,0.267477,0.012158,0.0,0.088146,0.571429,0.000000,0.0,0.012158,0.000000,0.012158
1,0.005147,0.195591,0.005147,0.0,0.366904,0.416917,0.005147,0.0,0.000000,0.005147,0.000000
2,0.034914,0.302589,0.000000,0.0,0.092232,0.546989,0.000000,0.0,0.011638,0.011638,0.000000
3,0.006798,0.237933,0.006798,0.0,0.360979,0.373895,0.000000,0.0,0.006798,0.006798,0.000000
4,0.031562,0.368227,0.000000,0.0,0.084692,0.494477,0.000000,0.0,0.010521,0.000000,0.010521


### Selecting features
Now we will select the features that we will use to build the model. We will use the `SelectKBest` method from the `sklearn` library to select the best features.

In [363]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

best_features = SelectKBest(score_func=chi2, k=5)
fit = best_features.fit(X, y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
feature_scores = pd.concat([dfcolumns, dfscores], axis=1)
feature_scores.columns = ['Feature', 'Score']
feature_scores = feature_scores.sort_values(by='Score', ascending=False)
feature_scores

,Feature,Score
4,Fare,10.693433
1,Age,3.558354
9,Sex_female,1.223414
10,Sex_male,1.073307
0,Pclass,0.521455
5,Cabin,0.303734
8,Embarked_S,0.171158
3,Parch,0.098493
6,Embarked_C,0.088387
2,SibSp,0.057086


### Model building
Now we will build the model. We will use the Perceptron and the Random Forest Classifier.

#### Perceptron

In [364]:
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score

perceptron = Perceptron()
perceptron.fit(X_train, y_train)
y_pred = perceptron.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.39664804469273746

#### Random Forest Classifier

In [365]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier()
random_forest.fit(X_train, y_train)
y_pred = random_forest.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.8547486033519553